## Cross-Dataset Analysis

In [1]:
dataset_id = " reporting"
!bq --location=US mk --dataset {dataset_id}

Dataset 'studied-brand-266702:reporting' successfully created.


### Implementation of cross-dataset queries

#### Query #1: Compare number of adverse events grouped by age, through a full outer join between drug dataset (dataset 2) and vaccine dataset (dataset 1)  

In [55]:
%%bigquery
select faers.Age, Drug_Cases, Vaccine_Cases
from 
(select AGE_YRS as Age, count(DISTINCT p.CASE_ID) as Drug_Cases
from faers_modeled.Patient_SQL_Final p inner join faers_modeled.Adverse_Event e on p.CASE_ID = e.CASE_ID
where COUNTRY = 'US'
group by AGE_YRS) faers
full join
(select AGE_YRS as Age, count(DISTINCT p.VAERS_ID) as Vaccine_Cases
from vaers_modeled.Patient p inner join vaers_modeled.Adverse_Event e on p.VAERS_ID = e.VAERS_ID
group by AGE_YRS) vaers
on faers.Age = vaers.Age
where faers.Age is not null
order by Age

,Age,Drug_Cases,Vaccine_Cases
0,0,428,957.0
1,1,275,1733.0
2,2,321,671.0
3,3,249,302.0
4,4,213,888.0
...,...,...,...
101,101,5,NaN
102,102,2,1.0
103,103,1,1.0
104,106,1,NaN


#### Create database view from cross-dataset query above (Age vs Drug_Cases and Vaccine_Cases) for Google Data Studio.

In [60]:
%%bigquery
create or replace view reporting.Cases_By_Age as
select faers.Age, Drug_Cases, Vaccine_Cases
from 
(select AGE_YRS as Age, count(DISTINCT p.CASE_ID) as Drug_Cases
from studied-brand-266702.faers_modeled.Patient_SQL_Final p inner join studied-brand-266702.faers_modeled.Adverse_Event e on p.CASE_ID = e.CASE_ID
where COUNTRY = 'US'
group by AGE_YRS) faers
full join
(select AGE_YRS as Age, count(DISTINCT p.VAERS_ID) as Vaccine_Cases
from studied-brand-266702.vaers_modeled.Patient p inner join studied-brand-266702.vaers_modeled.Adverse_Event e on p.VAERS_ID = e.VAERS_ID
group by AGE_YRS) vaers
on faers.Age = vaers.Age
where faers.Age is not null
order by Age

""
